In [14]:
import re
import string
import itertools as itools
import heapq as hq

class Maze:
    
    def __init__(self, dictionary):
        
        self.maze = dictionary
        self.adjacent = [1, -1, 1j, -1j]
        self.keys = {self.maze[x]: x for x in self.maze if self.maze[x] in string.ascii_lowercase}
        
        for pos, elem in self.maze.items():
            if elem == '@':
                self.entry = pos
                break
        
        return

    # Wrapper to access a given coordinate location. Returns 'None' if location is out of bound
    def access_loc(self, coord):
        
        found = None
        try:
            found = self.maze[coord]
        
        except:
            pass
        
        return found
        
    # Returns list of neighbours of given point in the form [[coordinates], [items]], 
    # where 'coordinates' and 'items' are in 1-to-1 correspondence.
    # kwarg 'select' is a regular expression that can be used to limit the search to
    # a given type of object. If not specified, all neighbors are returned.
    def near_neighbors(self, point, select=None):
        
        neighbor_list = [[], []]
        world = string.ascii_letters+'.'+'#'+'@'
        
        for adj in self.adjacent:
            
            neighbor = point+adj
            obj = self.access_loc(neighbor)
            if obj is not None:
                
                if select is None:
                    neighbor_list[0].append(neighbor)
                    neighbor_list[1].append(obj)
                    
                elif obj in re.findall(select, world):
                    neighbor_list[0].append(neighbor)
                    neighbor_list[1].append(obj)

                        
        return neighbor_list
    
    
    # This is not necessary, just a funny old thing that could be useful.
    # Fills all the dead ends of the maze with '#', effectively creating a new maze
    # which does not have dead ends. Could potentially solve
    def fill_dead_ends(self):
        
        complete = False
        
        while complete != True:
            
            complete = True
            
            for point in self.maze:
                
                if self.maze[point] != '.':
                    if self.maze[point] not in string.ascii_uppercase:
                        continue
                
                if self.near_neighbors(point)[1].count('#') >= 3:
                    self.maze[point] = '#'
                    complete = False
                    
        return
    
    
    def stringify(self):
        
        maze_width = int(max([x for x in self.maze if isinstance(x, int)]))
        y = 0
        maze_str = ''
        
        for n in range(len(self.maze)):

            x = n%(maze_width+1)
            maze_str += self.maze[x+y]
    
            if x == maze_width:
                y -= 1j
                x = 0
                maze_str += '\n'
    
            
        return maze_str
    
    
    # Calculates the (shortest) distance between any two keys in the maze, and also returns the
    # list of keys and doors encountered on the path from one to the other
    def distance(self, a, b):
        
        if a == b:
            return 0, []
                
        pos_a = self.entry if a == '@' else self.keys[a]
        pos_b = self.entry if b == '@' else self.keys[b]
        
        # Initialize list of heads, visited positions and found keys
        heads = [pos_a]
        found = {pos_a: [(a, 0)]}        
        #{pos_a: a}#{pos_a: [(a, 0)]} 
        visited = [pos_a]
        d = 0
        
        while heads != []:
            
            d +=1            
            new_heads = []

            for head in heads:
                                                        #'+keys.upper()+found[head].upper()+'
                accessible = self.near_neighbors(head, select='[a-zA-Z@\.]')
                for i, x in enumerate(accessible[0]):
                    if x not in visited:
                        found[x] = list(found[head])
                        
                        if accessible[1][i] in (string.ascii_letters+'@'):
                            #found[x] += accessible[1][i]
                            found[x].append((accessible[1][i], d))
                            #found[x][1] = d+1
                        
                        new_heads.append(x)
                        visited.append(x)
                             
                del found[head]
            
            heads = new_heads[:]
            
            if pos_b in heads:
                break
            
        
        return found[pos_b]
        
        
    # Modified Dijkstra algorithm
    def dij_solver(self, rmap, quadr):
    
        # Unique identifier for each entry of the heap, in case entries have equal lengths
        unique = itools.count()
        # Initialize the heap as a list of tuples with (distance, uniqueID, current node, keys collected)
        heap = [(0, next(unique), ('0', '1', '2', '3'), set(('0', '1', '2', '3')))]#('1', '2', '3', '4')
        # Stores the visited nodes
        G = {}
        
        while heap:
           
            # Pop head from heap, the node with current minimum distance
            curr_dist, dump, node, keys = hq.heappop(heap)
            # Success statement
            if keys >= set(self.keys):
                return curr_dist, node
            
            for new_key in set(self.keys) - set(keys):
                
                new_node = tuple(x if i != quadr[new_key] else new_key for i,x in enumerate(node))
                curr_key = node[quadr[new_key]]
                
                new_keys = set(keys)
                d, found = rmap[frozenset((curr_key, new_key))]
                doors = set([x.lower() for x in found if x.isupper()])
                new_keys.update([x for x in found if x.islower()])
            
                if new_keys >= doors:
                    
                    try:
                        old_dist = G[(new_node, frozenset(new_keys))]
                        if old_dist > (curr_dist + d):
                            G[(new_node, frozenset(new_keys))] = curr_dist + d
                            hq.heappush(heap, (curr_dist+d, next(unique), new_node, new_keys))
                                    
                    except:
                        G[(new_node, frozenset(new_keys))] = curr_dist + d
                        hq.heappush(heap, (curr_dist+d, next(unique), new_node, new_keys))
                                
        
        return 
    

In [15]:
x = 0
y = 0
m = dict()

with open('input.txt', 'r') as infile:
    for line in infile:
        symb = list(line.replace('\n', ''))
        for s in symb:
            m[x+y] = s
            x +=1
        x = 0 
        y -= 1j
        
# Initialize Maze object
maze = Maze(m)
# Fill dead ends
maze.fill_dead_ends()
# Uncomment to see the new graph with no dead ends 
#print(maze.stringify())

roadmap = {}
# All possible pair of keys in the maze
pairs = [frozenset((u, v)) for u, v in itools.combinations(''.join(maze.keys)+'@', 2)]

#print(pairs)
while len(pairs)>0:
       
    u, v = pairs[-1]
    
    # Returns a list of tuples [(u, d1=0), (a, d2), ..., (v, dN)]
    # where {u, a, ..., v} can be both doors or keys, and {d1, d2, ..., dN}
    # are the distances from the first key u. The list is already sorted.
    found = maze.distance(u, v)
    
    #
    for i in range(len(found)-1):
        k1, dist1 = found[i]
        
        if k1.isupper():
            continue
        
        accum = k1
        
        for j in range(i+1, len(found)):
            k2, dist2 = found[j]
         
            if k2.isupper() or (frozenset((k1, k2)) in roadmap): 
                accum+=k2
                continue
                
            pairs.remove(frozenset((k1, k2)))
            accum += k2
            rel = abs(dist1-dist2)
            roadmap[frozenset((k1, k2))] = (abs(dist1-dist2), set(accum.replace('@', '')))
            
#print(len(roadmap))

#################################################################################
###.........###################.....###########...#...###...###...Q...........###
###.#######.###################.###.###########.#.#.#.###.#.###.#######.#####.###
#...#######.................###.###.....#####...#z#.#...#.#.###...#u..#....b#...#
#.#########################.###.#######.#####.#####.###.#.#.#####.#.#.#########.#
#.............#####l..#####...#.....###.#####...#...#...#.#...###.#.#...#####...#
#############.#######.#######.#####.###.#######.#.###.###.###.###.#.###.#####.###
#############.#######.#######.#.....#...#.......#.###.###...#...#.#...#.......###
#############.#######.#######.#.#####.###.#######.###.#####.###.#.###.###########
#############.#######.....###.#.....#...#.#######...#...#...#...#...#...#.......#
#############.###########.###.#####.###.#.#########.###.#.###.#####.###.#.#####.#
#############.###########.###.#...#.###.#.###.......###...###.###...###...###.A.#
#############.##

In [16]:
#print(roadmap)
entry_pos = complex(maze.entry)
four_mazes = {}

# Initializing the four_mazes variable
for key, val in maze.keys.items():
    
    if val.real>entry_pos.real and val.imag> entry_pos.imag:
        quad = 0        
    elif val.real>entry_pos.real and val.imag < entry_pos.imag:
        quad = 1
    elif val.real<entry_pos.real and val.imag > entry_pos.imag:
        quad = 2
    elif val.real<entry_pos.real and val.imag < entry_pos.imag:
        quad = 3

    four_mazes[key] = quad
    old_map = roadmap.pop(frozenset(('@', key)), None)
    roadmap[frozenset((str(quad), key))] = (old_map[0]-2, old_map[1])
    
    
print(four_mazes)

{'z': 0, 'u': 0, 'b': 0, 'l': 2, 'm': 0, 'w': 0, 'e': 2, 'i': 0, 'y': 0, 'x': 0, 'v': 0, 'f': 0, 'o': 0, 'k': 0, 'j': 0, 'p': 0, 'g': 1, 's': 1, 't': 3, 'r': 3, 'd': 1, 'c': 3, 'q': 1, 'a': 3, 'n': 3, 'h': 3}


In [17]:
minimum, final_node = maze.dij_solver(roadmap, four_mazes)
print(minimum, final_node)

1886 ('i', 'q', 'e', 'r')
